In [28]:
import h5py
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [29]:
file_predictions = 'predictions.txt'
train_file = 'MA0035_4_m5_train.h5'
test_file = 'MA0035_4_m5_test.h5'

In [30]:
h5_train = h5py.File(train_file, 'r')
h5_test = h5py.File(test_file, 'r')

In [95]:
train_data = h5_train['data']
train_binlabels = h5_train['binlabels']

train_data = train_data[:20000]
train_binlabels = train_binlabels[:20000]

(100000, 4, 1000)


In [96]:
print(train_data.shape)
print(train_binlabels.shape)
print(train_data[0])
print(train_binlabels[0])

(20000, 4, 1000)
(20000, 1)
[[0 1 1 ... 0 1 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 1]
 [0 0 0 ... 0 0 0]]
[0]


In [110]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(4, 1000)),
        tf.keras.layers.Dense(512, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(512, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(32, activation=tf.keras.activations.relu),
        tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid)
    ]
)

In [111]:
model.compile(
    optimizer = tf.optimizers.Adam(),
    loss = 'binary_crossentropy',
    metrics=['accuracy']
)

In [112]:
history = model.fit(train_data, train_binlabels, epochs=15)

Epoch 1/15
625/625 [==============================] - 5s 8ms/step - loss: 0.6332 - accuracy: 0.6856
Epoch 2/15
625/625 [==============================] - 5s 8ms/step - loss: 0.6055 - accuracy: 0.6930
Epoch 3/15
625/625 [==============================] - 5s 8ms/step - loss: 0.5892 - accuracy: 0.6994
Epoch 4/15
625/625 [==============================] - 5s 8ms/step - loss: 0.5682 - accuracy: 0.7087
Epoch 5/15
625/625 [==============================] - 5s 8ms/step - loss: 0.5546 - accuracy: 0.7206
Epoch 6/15
625/625 [==============================] - 5s 8ms/step - loss: 0.4961 - accuracy: 0.7707
Epoch 7/15
625/625 [==============================] - 6s 9ms/step - loss: 0.4206 - accuracy: 0.8165
Epoch 8/15
625/625 [==============================] - 5s 9ms/step - loss: 0.2935 - accuracy: 0.8843
Epoch 9/15
625/625 [==============================] - 5s 8ms/step - loss: 0.2007 - accuracy: 0.9273
Epoch 10/15
625/625 [==============================] - 5s 9ms/step - loss: 0.1297 - accuracy: 0.9577

In [113]:
test_data = h5_test['data']
test_binlabels = h5_test['binlabels']

In [114]:
loss, acc = model.evaluate(test_data[:5000])

print(loss)
print(acc)

157/157 [==============================] - 0s 354us/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
0.0
0.0


In [115]:
for i in range(100):
    print(test_binlabels[i], end='[')
    print(model.predict(np.array([test_data[i]])), end=']\n')
# model.predict(test_data[:1000])

[1][[[9.33853e-06]]]
[1][[[0.01023421]]]
[0][[[0.06701809]]]
[0][[[0.00130472]]]
[1][[[8.820681e-06]]]
[0][[[0.0822435]]]
[1][[[0.00050902]]]
[0][[[0.9923306]]]
[0][[[8.947367e-06]]]
[0][[[0.00575572]]]
[0][[[3.2411353e-05]]]
[0][[[0.14241171]]]
[0][[[0.00096044]]]
[1][[[0.2160233]]]
[0][[[0.00011185]]]
[0][[[0.00555944]]]
[1][[[0.00251675]]]
[0][[[0.00754455]]]
[1][[[0.00033236]]]
[0][[[0.00671384]]]
[0][[[3.261396e-08]]]
[0][[[0.04448059]]]
[0][[[0.00111681]]]
[0][[[0.00083876]]]
[0][[[0.0007149]]]
[0][[[0.00302827]]]
[0][[[0.03073025]]]
[0][[[0.00201869]]]
[0][[[0.8828747]]]
[0][[[0.00043675]]]
[0][[[0.00129852]]]
[1][[[0.65104103]]]
[1][[[7.240207e-07]]]
[0][[[0.24851137]]]
[0][[[1.819602e-05]]]
[0][[[0.00082821]]]
[0][[[0.891557]]]
[0][[[5.601827e-08]]]
[0][[[0.71866167]]]
[0][[[0.00136411]]]
[0][[[0.00245255]]]
[0][[[0.6300951]]]
[0][[[0.0800387]]]
[0][[[0.9943925]]]
[0][[[0.979234]]]
[0][[[0.13732454]]]
[0][[[0.00619799]]]
[0][[[0.00035733]]]
[0][[[0.00086269]]]
[0][[[0.557871]]